In [8]:
import numpy as np
import scipy.stats as sps
from scipy.stats import norm

In [17]:
def gibbs_hyper(thet, nbeps2, var):
    # Gibbs step for the hyperparameter, with uniform prior
    res = np.random.uniform(-4, 4, nbeps2)
    test = np.mean(np.random.normal(res, var, (len(thet), nbeps2)), axis=0)
    dist = np.abs(test - np.mean(thet))
    return res[np.argmin(dist)], np.min(dist)

def gibbs_step_param(data, hyper, var, sigm, M, qq):
    # Gibbs step for the parameters
    resparam = np.random.normal(hyper, var, (len(data), M * qq))
    test = np.mean(np.random.normal(resparam.reshape(-1), sigm, (len(data), M)), axis=1)
    dist = np.abs(data - test)
    weights = np.exp(-dist)  # Calcul des poids
    weights = weights / np.sum(weights)  # Normalisation des poids
    return resparam, np.sum(dist), weights  # Retourne également les poids

def gibbstot(data, simu, poids, rprior, histep, alpha, sigm, var, M, nbeps2, n, qq):
    # Full function
    simu = np.empty((len(simu), n + 1))
    poids = np.empty((len(poids), n + 1))
    dist = np.empty(n)
    weights = []
    eps_hist = []
    hyper = np.empty(n + 1)

    simu[:, 0] = simu
    poids[:, 0] = poids
    dist[0] = rprior
    hyper[0] = alpha
    weights.append(poids)  # Stockage des poids initiaux
    eps_hist.append(histep)

    for i in range(1, n + 1):
        VV = gibbs_step_param(data, simu[:, i - 1], var, sigm, M, qq)
        simu[:, i], dist[i - 1], w = VV  # Récupération des poids
        poids[:, i] = w  # Stockage des poids
        WW = gibbs_hyper(simu[:, i], nbeps2, var)
        hyper[i] = WW[0]
        eps_hist.append(WW[0])
    
    return simu, poids, dist, weights, eps_hist, hyper

In [19]:
# Données et fonctions de statistiques

l = 50
data = np.random.normal(-5, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu, means):
    return np.sum(norm.pdf(simu, loc=means, scale=1))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i], 1).ppf(lev[j]) for j in range(9)] for i in range(20)])
    return np.sum(abs(quantiles - statdata))

In [21]:
# Utilisation des données pour l'algorithme Gibbs ABC
resultat_gibbsABC = gibbstot(data, 50, 50, 50, 150, 0.9, rprior, dprior, dist, l, 50, 10)
print(resultat_gibbsABC)

# Utilisation des mêmes paramètres que ceux utilisés pour l'algorithme SMC ABC
M = 50
nbeps2 = 50
var = 1
sigm = 1
qq = 50
n = 150

# Exécution de l'algorithme Gibbs ABC avec les mêmes paramètres
resultat_gibbsABC = gibbstot(data, simuini, poidsini, distini, histepsini, hyperini, sigm, var, M, nbeps2, n, qq)
print(resultat_gibbsABC)


TypeError: object of type 'int' has no len()